# Exercise 8
Load the MNIST data (introduced in Chapter 3), and split it into a training set, a
validation set, and a test set (e.g., use 50,000 instances for training, 10,000 for validation,
and 10,000 for testing). Then train various classifiers, such as a Random
Forest classifier, an Extra-Trees classifier, and an SVM. Next, try to combine
them into an ensemble that outperforms them all on the validation set, using a
soft or hard voting classifier. Once you have found one, try it on the test set. How
much better does it perform compared to the individual classifiers?

In [1]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml("mnist_784", version = 1) # divided into ["data"] and ["target"]

Subset mnist dataset:

In [3]:
tr, vl, ts = 50000, 60000, 70000
train, val, test = [x for x in range(tr)], [x for x in range(tr, vl)], [x for x in range(vl, ts)]
X_train, X_val, X_test = mnist["data"][train], mnist["data"][val], mnist["data"][test]
y_train, y_val, y_test = mnist["target"][train], mnist["target"][val], mnist["target"][test]

Train classifiers: Random Forest, Boosted Trees, SVC

In [2]:
# imports
import time
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost
from sklearn.metrics import precision_score, recall_score

In [5]:
X_train[0]

array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   

## SVM Classifier
Since we are dealing with a medium/large dataset, we will use kernel approximation in combination with stochastic gradient descent linear classifier (that natively handles multiclass problems)

https://scikit-learn.org/stable/modules/kernel_approximation.html

In [ ]:
# SVC
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# pipeline: X --> standardscaler --> RBFSampler --> SGDClassifier

    
"""    
time0 = time.time()
svm_clf = SVC
svm_clf.fit(X_train, y_train)
time1 = time.time()
print("Training time: {}".format(time1 - time0))
"""